<a href="https://colab.research.google.com/github/Sagargomad/My_projects/blob/main/CNN_network_from_scratch_using_tensorflow_on_transformed_data%2Con_Fashion_MNIST_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split


In [ ]:
# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Split the training dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Function to preprocess images
def preprocess_image(image):
    # Resize image to a quadratic shape (e.g., 64x64) using OpenCV
    image = cv2.resize(image, (64, 64))
    # Normalize pixel values to the range [0, 1]
    image = image / 255.0
    return image

# Apply preprocessing to all datasets
train_images = np.array([preprocess_image(img) for img in train_images])
val_images = np.array([preprocess_image(img) for img in val_images])
test_images = np.array([preprocess_image(img) for img in test_images])


4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images[..., np.newaxis], train_labels, epochs=10, validation_data=(val_images[..., np.newaxis], val_labels))

# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(model.history.history['val_accuracy'])

# Save the model with the highest validation accuracy
model.save(f'fashion_mnist_cnn_best_model_epoch_{best_epoch}.h5')


Epoch 1/10
1500/1500 [==============================] - 243s 161ms/step - loss: 0.4197 - accuracy: 0.8504 - val_loss: 0.3181 - val_accuracy: 0.8839
Epoch 2/10
1500/1500 [==============================] - 242s 162ms/step - loss: 0.2742 - accuracy: 0.8994 - val_loss: 0.2687 - val_accuracy: 0.9007
Epoch 3/10
1500/1500 [==============================] - 237s 158ms/step - loss: 0.2257 - accuracy: 0.9154 - val_loss: 0.2689 - val_accuracy: 0.9010
Epoch 4/10
1500/1500 [==============================] - 248s 165ms/step - loss: 0.1862 - accuracy: 0.9312 - val_loss: 0.2421 - val_accuracy: 0.9106
Epoch 5/10
1500/1500 [==============================] - 236s 158ms/step - loss: 0.1538 - accuracy: 0.9421 - val_loss: 0.2486 - val_accuracy: 0.9153
Epoch 6/10
1500/1500 [==============================] - 239s 159ms/step - loss: 0.1281 - accuracy: 0.9522 - val_loss: 0.2425 - val_accuracy: 0.9159
Epoch 7/10
1500/1500 [==============================] - 240s 160ms/step - loss: 0.1009 - accuracy: 0.9619 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Split the training dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Function to preprocess and augment images
def preprocess_and_augment(image):
    # Resize image to a quadratic shape (e.g., 64x64) using OpenCV
    image = cv2.resize(image, (64, 64))
    # Add an extra channel to make it compatible with ImageDataGenerator
    image = np.expand_dims(image, axis=-1)
    # Normalize pixel values to the range [0, 1]
    image = image / 255.0
    return image

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Apply preprocessing and augmentation to all datasets
train_images = np.array([preprocess_and_augment(img) for img in train_images])
val_images = np.array([preprocess_and_augment(img) for img in val_images])
test_images = np.array([preprocess_and_augment(img) for img in test_images])

# Define and train the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(model.history.history['val_accuracy'])

# Save the model with the highest validation accuracy
model.save(f'fashion_mnist_cnn_best_model_epoch_{best_epoch}.h5')

# Use the saved model for inference
loaded_model = tf.keras.models.load_model(f'fashion_mnist_cnn_best_model_epoch_{best_epoch}.h5')

# Make predictions on test data
predictions = loaded_model.predict(test_images)

# Evaluate the model on test data
test_loss, test_accuracy = loaded_model.evaluate(test_images, test_labels)

print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/10
1500/1500 [==============================] - 248s 165ms/step - loss: 0.4167 - accuracy: 0.8511 - val_loss: 0.3169 - val_accuracy: 0.8827
Epoch 2/10
1500/1500 [==============================] - 242s 161ms/step - loss: 0.2773 - accuracy: 0.8995 - val_loss: 0.2714 - val_accuracy: 0.8988
Epoch 3/10
1500/1500 [==============================] - 243s 162ms/step - loss: 0.2276 - accuracy: 0.9161 - val_loss: 0.2532 - val_accuracy: 0.9097
Epoch 4/10
1500/1500 [==============================] - 222s 148ms/step - loss: 0.1913 - accuracy: 0.9282 - val_loss: 0.2400 - val_accuracy: 0.9132
Epoch 5/10
1500/1500 [==============================] - 223s 149ms/step - loss: 0.1559 - accuracy: 0.9415 - val_loss: 0.2468 - val_accuracy: 0.9160
Epoch 6/10
1500/1500 [==============================] - 224s 150ms/step - loss: 0.1331 - accuracy: 0.9508 - val_loss: 0.2539 - val_accuracy: 0.9124
Epoch 7/10
1500/1500 [==============================] - 232s 155ms/step - loss: 0.1080 - accuracy: 0.9602 - val_

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Split the training dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Function to preprocess and augment images
def preprocess_and_augment(image):
    # Resize image to a quadratic shape (e.g., 64x64) using OpenCV
    image = cv2.resize(image, (64, 64))
    # Add an extra channel to make it compatible with ImageDataGenerator
    image = np.expand_dims(image, axis=-1)
    # Normalize pixel values to the range [0, 1]
    image = image / 255.0
    return image

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Apply preprocessing and augmentation to all datasets
train_images = np.array([preprocess_and_augment(img) for img in train_images])
val_images = np.array([preprocess_and_augment(img) for img in val_images])
test_images = np.array([preprocess_and_augment(img) for img in test_images])

# Define and train the CNN model with Batch Normalization and Dropout
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks for ReduceLRonPlateau and Early Stopping
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with callbacks
history = model.fit(train_images, train_labels, epochs=50, validation_data=(val_images, val_labels), callbacks=[reduce_lr, early_stopping])

# Use the saved model for inference
loaded_model = model

# Make predictions on test data
predictions = loaded_model.predict(test_images)

# Evaluate the model on test data
test_loss, test_accuracy = loaded_model.evaluate(test_images, test_labels)

print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/50
1500/1500 [==============================] - 338s 222ms/step - loss: 0.8869 - accuracy: 0.6841 - val_loss: 0.4738 - val_accuracy: 0.8224 - lr: 0.0010
Epoch 2/50
1500/1500 [==============================] - 332s 221ms/step - loss: 0.6044 - accuracy: 0.7672 - val_loss: 0.3859 - val_accuracy: 0.8721 - lr: 0.0010
Epoch 3/50
1500/1500 [==============================] - 338s 225ms/step - loss: 0.5135 - accuracy: 0.8028 - val_loss: 0.3414 - val_accuracy: 0.8737 - lr: 0.0010
Epoch 4/50
1500/1500 [==============================] - 332s 221ms/step - loss: 0.4589 - accuracy: 0.8251 - val_loss: 0.3427 - val_accuracy: 0.8876 - lr: 0.0010
Epoch 5/50
1500/1500 [==============================] - 328s 219ms/step - loss: 0.4217 - accuracy: 0.8391 - val_loss: 0.3239 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 6/50
1500/1500 [==============================] - 323s 215ms/step - loss: 0.3945 - accuracy: 0.8506 - val_loss: 0.2979 - val_accuracy: 0.8973 - lr: 0.0010
Epoch 7/50
1500/1500 [============